# Multi-Tier Web Application

In this exercise we will be using OpenTofu (an open-source fork of Terraform) and Oracle Cloud Infrastructure to deploy a multi-tier web application using a VM instance and a Oracel Autonomous Database.

[OpenTofu](https://opentofu.org)

[Oracle Cloud Free Tier](https://www.oracle.com/au/cloud/free/)

In [ ]:
!oci session refresh --profile harleycalvert

# Set Up OCI Terraform

[Set Up OCI Terraform](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-provider/01-summary.htm)

## Install OpenTofu

In [ ]:
!snap install --classic opentofu

In [ ]:
!tofu -v

## Create RSA Keys

In [ ]:
!mkdir $HOME/.oci

In [ ]:
!openssl genrsa -out $HOME/.oci/oci_rsa_key.pem 2048

In [ ]:
!chmod 600 $HOME/.oci/oci_rsa_key.pem

In [ ]:
!openssl rsa -pubout -in $HOME/.oci/oci_rsa_key.pem -out $HOME/.oci/oci_rsa_key.pem.pub

In [ ]:
!(cd $HOME/.oci/ && ls)

In [ ]:
!cat $HOME/.oci/oci_rsa_key.pem.pub

Add the public key to your user account.

In the OCI Console's top navigation bar, click the Profile menu, and then go to My profile.
- Click API Keys.
- Click Add API Key.
- Select Paste Public Keys.
- Paste value from previous step, including the lines with BEGIN PUBLIC KEY and END PUBLIC KEY.
- Click Add.
- Copy the Configuration file preview.
- Paste the Configuration file preview into ~/.oci/config

You have now set up the RSA keys to connect to your OCI account.

## Add API Key-Based Authentication

In [ ]:
!mkdir tf-provider

## Gather Required Information
Collect the following credential information from the OCI Console.

- Tenancy OCID
- User OCID
- Fingerprint
- Region 
  - ap-melbourne-1

Collect the following information from your environment.

- Private Key Path 
  - /home/harley/.oci/oci_rsa_key.pem
  
Put the information in provider.tf like so:

In [ ]:
%%writefile ./tf-provider/provider.tf

provider "oci" {  
  tenancy_ocid = "<tenancy-ocid>"
  user_ocid = "<user-ocid>" 
  private_key_path = "<rsa-private-key-path>"
  fingerprint = "<fingerprint>"
  region = "<region-identifier>"
}

In [ ]:
!code ./tf-provider/provider.tf

In [ ]:
%%writefile ./tf-provider/availability-domains.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/data-sources/identity_availability_domains

# Tenancy is the root or parent to all compartments.
# For this, use the value of <tenancy-ocid> for the compartment OCID.

data "oci_identity_availability_domains" "ads" {
  compartment_id = "<tenancy-ocid>"
}

In [ ]:
!code ./tf-provider/availability-domains.tf

In [ ]:
%%writefile ./tf-provider/outputs.tf


# Output the "list" of all availability domains.
output "all-availability-domains-in-your-tenancy" {
  value = data.oci_identity_availability_domains.ads.availability_domains
}

In [ ]:
!code ./tf-provider/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-provider && tofu init) 

In [ ]:
!(cd tf-provider && ls -al) 

In [ ]:
!(cd tf-provider && tree) 

In [ ]:
!(cd tf-provider && tofu validate)

In [ ]:
!(cd tf-provider && tofu plan -input=false)

In [ ]:
!(cd tf-provider && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-provider && tofu output)

# Create a Compartment

[Create a Compartment](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-compartment/01-summary.htm)

## Create Scripts

In [ ]:
!mkdir ./tf-compartment

In [ ]:
!cp ./tf-provider/provider.tf ./tf-compartment/provider.tf 

In [ ]:
%%writefile ./tf-compartment/compartment.tf


resource "oci_identity_compartment" "tf-compartment" {
    # Required
    compartment_id = "<tenancy-ocid>"
    description = "<your-description>"
    name = "<your-compartment-name>"
}

Replace your-compartment-name with a name of your choice.

In [ ]:
!code ./tf-compartment/compartment.tf

In [ ]:
%%writefile ./tf-compartment/outputs.tf


# Outputs for compartment
output "compartment-name" {
  value = oci_identity_compartment.tf-compartment.name
}

output "compartment-OCID" {
  value = oci_identity_compartment.tf-compartment.id
}

In [ ]:
!code ./tf-compartment/outputs.tf

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-compartment && ls -al) 

In [ ]:
!(cd tf-compartment && tofu init) 

In [ ]:
!(cd tf-compartment && tofu validate)

In [ ]:
!(cd tf-compartment && tofu plan -input=false)

In [ ]:
!(cd tf-compartment && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-compartment && tofu output)

# Create a Virtual Cloud Network

[Create a Virtual Cloud Network](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-vcn/01-summary.htm)

## Create a Basic Network

In [ ]:
!mkdir ./tf-vcn

In [ ]:
!cp ./tf-provider/provider.tf ./tf-vcn/provider.tf 

In [ ]:
%%writefile ./tf-vcn/vcn.tf


# https://github.com/oracle-terraform-modules/terraform-oci-vcn

resource "oci_core_vcn" "vcn" {
  # We still allow module users to declare a cidr using `vcn_cidr` instead of the now recommended `vcn_cidrs`, but internally we map both to `cidr_blocks`
  # The module always use the new list of string structure and let the customer update his module definition block at his own pace.
  cidr_blocks    = ["10.0.0.0/16"]
  compartment_id = var.compartment_id
  display_name   = "vcn"
  dns_label      = "vcn"
  is_ipv6enabled = false

  freeform_tags = {
    terraformed = "Please do not edit manually"
    module      = "oracle-terraform-modules/vcn/oci"
  }
    
  defined_tags  = null

  lifecycle {
    ignore_changes = [defined_tags, dns_label, freeform_tags]
  }
}

## Customise the Network
### Create a Security List for the Public Subnet

In [ ]:
%%writefile ./tf-vcn/public-security-list.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_security_list

resource "oci_core_security_list" "public-security-list"{

# Required
  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id

# Optional
  display_name = "security-list-for-public-subnet"
    
  
  egress_security_rules {
    stateless = false
    destination = "0.0.0.0/0"
    destination_type = "CIDR_BLOCK"
    protocol = "all" 
  }

 
  ingress_security_rules {
    stateless   = false
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"

    # Allow SSH (TCP port 22)
    protocol = "6"  # TCP
    tcp_options {
      min = 22
      max = 22
    }
  }

    
  ingress_security_rules {
    stateless   = false
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"

    # Allow HTTP (TCP port 80)
    protocol = "6"  # TCP
    tcp_options {
      min = 80
      max = 80
    }
  }

    
  ingress_security_rules {
    stateless   = false
    source      = "0.0.0.0/0"
    source_type = "CIDR_BLOCK"

    # Allow HTTPS (TCP port 443)
    protocol = "6"  # TCP
    tcp_options {
      min = 443
      max = 443
    }
 }  


}

### Create a Public Subnet

In [ ]:
%%writefile ./tf-vcn/public-subnet.tf


# Source from https://registry.terraform.io/providers/oracle/oci/latest/docs/resources/core_subnet
# https://docs.oracle.com/en-us/iaas/tools/terraform-provider-oci/5.24/docs/r/core_subnet.html

resource "oci_core_subnet" "vcn-public-subnet"{

  # Required
  compartment_id = var.compartment_id
  vcn_id         = oci_core_vcn.vcn.id
  cidr_block     = "10.0.0.0/24"
 
  # Optional
  route_table_id      = oci_core_route_table.public-route-table.id
  security_list_ids   = [oci_core_security_list.public-security-list.id]
  display_name        = "public-subnet"
  dns_label           = "public"
}

### Create Internet Gateway

In [ ]:
%%writefile ./tf-vcn/internet-gateway.tf


resource "oci_core_internet_gateway" "internet-gateway" {
  compartment_id = var.compartment_id
  display_name = "internet-gateway"
  vcn_id = oci_core_vcn.vcn.id
}

### Create Public Subnet Route Table

In [ ]:
%%writefile ./tf-vcn/public-route-table.tf


resource "oci_core_route_table" "public-route-table" {

  compartment_id = var.compartment_id
  vcn_id = oci_core_vcn.vcn.id
  display_name = "public-route-table"

  route_rules {
    destination_type = "CIDR_BLOCK"
    destination = "0.0.0.0/0"
    network_entity_id = oci_core_internet_gateway.internet-gateway.id
  }
}

### Variables

In [ ]:
%%writefile ./tf-vcn/variables.tf


variable "compartment_id" {
  # This is the ID of the compartment you created earlier.
  # Check your compartment output  
  description = "Compartment ID"    
  type        = string
  default     = "<compartment-ocid>"
}

In [ ]:
!code ./tf-vcn/variables.tf

### Outputs

In [ ]:
%%writefile ./tf-vcn/outputs.tf
 

# Outputs for public security list
output "public-security-list-name" {
  value = oci_core_security_list.public-security-list.display_name
}
output "public-security-list-OCID" {
  value = oci_core_security_list.public-security-list.id
}


# Outputs for public subnet
output "public-subnet-name" {
  value = oci_core_subnet.vcn-public-subnet.display_name
}
output "public-subnet-OCID" {
  value = oci_core_subnet.vcn-public-subnet.id
}


## Run Scripts

In [14]:
!pwd

/home/harley/cloud/oci-notebook-adb


In [ ]:
!(cd tf-vcn && ls -al) 

In [ ]:
!(cd tf-vcn && tofu init) 

In [ ]:
!(cd tf-vcn && tofu validate)

In [ ]:
!(cd tf-vcn && tofu plan -input=false)

In [ ]:
!(cd tf-vcn && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-vcn && tofu output)

# Create an Autonomous Database

## Create Scripts

In [ ]:
!mkdir tf-database

In [ ]:
!cp ./tf-provider/provider.tf ./tf-database/provider.tf 

In [ ]:
%%writefile ./tf-database/database.tf

# https://docs.oracle.com/en-us/iaas/tools/terraform-provider-oci/5.24/docs/r/database_autonomous_database.html
# https://hiteshgondalia.wordpress.com/2020/01/15/launch-oci-free-tier-atp-and-adw-using-terraform-on-window-platform/

resource "random_string" "autonomous_database_admin_password" {
  length      = 28
  min_numeric = 1
  min_lower   = 1
  min_upper   = 1
  min_special = 1
}

resource "oci_database_autonomous_database" "autonomous_database" {
    
  # Required
  compartment_id           = "<compartment-ocid>"
  db_name                  = "<your-autonomous-database-name>"
    
  # Optional
  admin_password           = "${random_string.autonomous_database_admin_password.result}"
  cpu_core_count           = 1
  data_storage_size_in_tbs = 1
  display_name             = "<display-name>"
  is_free_tier             = true
  # subnet_id breaks the free tier compatibility
  # subnet_id                = "<oci_core_subnet.subnet-id>"
}


In [ ]:
!code ./tf-database/database.tf

In [ ]:
%%writefile ./tf-database/outputs.tf


output "autonomous_database_connection" {
  value = oci_database_autonomous_database.autonomous_database.connection_strings
}

output "autonomous_database_admin_password" {
  value = "${random_string.autonomous_database_admin_password.result}"
}

output "autonomous_database_high_connection_string" {
  value = "${lookup(oci_database_autonomous_database.autonomous_database.connection_strings.0.all_connection_strings, "high", "unavailable")}"
}


## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-database && ls -al) 

In [ ]:
!(cd tf-database && tofu init) 

In [ ]:
!(cd tf-database && tofu validate)

In [ ]:
!(cd tf-database && tofu plan -input=false)

In [ ]:
!(cd tf-database && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-database && tofu output)

## SQL

In the OCI console choose Database actions -->> SQL

```sql
CREATE USER bob IDENTIFIED BY EO750FAS7F6Tvpzx;
```

``` sql
CREATE TABLE accounts (
  user_id NUMBER GENERATED BY DEFAULT ON NULL AS IDENTITY,
  email VARCHAR2(50),
  password VARCHAR2(50),
  CONSTRAINT accounts_pk PRIMARY KEY (user_id)
);
```

``` sql
CREATE TABLE pets (
  pet_id NUMBER GENERATED BY DEFAULT ON NULL AS IDENTITY,
  user_id NUMBER NOT NULL,
  image BLOB NOT NULL,
  type VARCHAR2(255) NOT NULL,
  breed VARCHAR2(255) NOT NULL,
  dob DATE NOT NULL,
  CONSTRAINT pets_pk PRIMARY KEY (pet_id),
  CONSTRAINT fk_accounts FOREIGN KEY (user_id) 
  REFERENCES accounts (user_id) 
  ON DELETE CASCADE 
);
```

```sql
INSERT INTO accounts (email, password)
VALUES ('alice@here.com', 'd#n4vcwo2pfw4DSF6onKDSwn');
```

```sql
INSERT INTO accounts (email, password)
VALUES ('bob@here.com', 'd#n4vcwo2pfw4DSF6onKDSwn');
```

```sql
INSERT INTO accounts (email, password)
VALUES ('eve@here.com', 'd#n4vcwo2pfw4DSF6onKDSwn');
```

``` sql
GRANT SELECT, INSERT, UPDATE, DELETE ON ACCOUNTS TO BOB;
```

``` sql
GRANT SELECT, INSERT, UPDATE, DELETE ON PETS TO BOB;
```

# Create a Compute Instance

[Create a Compute Instance](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/tf-compute/01-summary.htm)

## Create SSH Encryption Keys

Execute the following command in the terminal:

In [ ]:
!ssh-keygen -t rsa -b 2048 -C "" -f ~/.ssh/oci_key

## Create Scripts

In [ ]:
!mkdir tf-compute

In [ ]:
!cp ./tf-provider/provider.tf ./tf-compute/provider.tf 

In [ ]:
!cp ./tf-provider/availability-domains.tf ./tf-compute/availability-domains.tf

In [ ]:
%%writefile ./tf-compute/compute.tf


resource "oci_core_instance" "webserver_instance" {
  # Required
  availability_domain = data.oci_identity_availability_domains.ads.availability_domains[0].name
  compartment_id = "<compartment-ocid>"
  shape = "VM.Standard.E2.1.Micro"
  source_details {
    # https://docs.oracle.com/en-us/iaas/images/ 
    # I used Canonical-Ubuntu-22.04-Minimal  
    source_id = "<source-ocid>"
    source_type = "image"
  }

  # Optional
  display_name = "<your-ubuntu-instance-name>"
  create_vnic_details {
    assign_public_ip = true
    # Public subnet
    subnet_id = "<subnet-ocid>"
  }
  metadata = {
    ssh_authorized_keys = file("<ssh-public-key-path>")
  } 
  preserve_boot_volume = false
}

In [29]:
!code ./tf-compute/compute.tf 

In [ ]:
%%writefile ./tf-compute/outputs.tf


# The "name" of the availability domain to be used for the compute instance.
output "name-of-first-availability-domain" {
  value = data.oci_identity_availability_domains.ads.availability_domains[0].name
}

# Outputs for compute instance
output "public-ip-for-compute-instance" {
  value = oci_core_instance.webserver_instance.public_ip
}

output "instance-name" {
  value = oci_core_instance.webserver_instance.display_name
}

output "instance-OCID" {
  value = oci_core_instance.webserver_instance.id
}

output "instance-region" {
  value = oci_core_instance.webserver_instance.region
}

output "instance-shape" {
  value = oci_core_instance.webserver_instance.shape
}

output "instance-state" {
  value = oci_core_instance.webserver_instance.state
}

output "instance-OCPUs" {
  value = oci_core_instance.webserver_instance.shape_config[0].ocpus
}

output "instance-memory-in-GBs" {
  value = oci_core_instance.webserver_instance.shape_config[0].memory_in_gbs
}

output "time-created" {
  value = oci_core_instance.webserver_instance.time_created
}

In [33]:
!code ./tf-compute/outputs.tf 

## Run Scripts

In [ ]:
!pwd

In [ ]:
!(cd tf-compute && ls -al) 

In [ ]:
!(cd tf-compute && tofu init) 

In [ ]:
!(cd tf-compute && tofu validate)

In [ ]:
!(cd tf-compute && tofu plan -input=false)

In [ ]:
!(cd tf-compute && tofu apply -input=false -auto-approve)

In [ ]:
!(cd tf-compute && tofu output)

## Connect to the Instance

Run the following command in the terminal:

```bash
ssh -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
```

## Set up Apache

[Install Apache and PHP on an Oracle Linux Instance](https://docs.oracle.com/en-us/iaas/developer-tutorials/tutorials/apache-on-oracle-linux/01-summary.htm)

Run the following command in the webserver terminal:

```bash
sudo su

apt update
apt -y install apache2
systemctl restart apache2

iptables -I INPUT 6 -m state --state NEW -p tcp --dport 80 -j ACCEPT
iptables -I INPUT 6 -m state --state NEW -p tcp --dport 443 -j ACCEPT
netfilter-persistent save
```

In the browser, connect to http://your-public-ip-address

## Set up PHP

Run the following command in the webserver terminal:

```bash
apt -y install php libapache2-mod-php
php -v
systemctl restart apache2

echo '<?php
  phpinfo();
?>' > /var/www/html/info.php
```

In the browser, connect to http://your-public-ip-address/info.php

## Connecting PHP to Oracle Database

[Oracle Instant Client Downloads for Linux x86-64 (64-bit)
](https://www.oracle.com/database/technologies/instant-client/linux-x86-64-downloads.html)

Run the following command in the webserver terminal:

```bash
sudo su
mkdir /opt/oracle
cd /opt/oracle 

wget https://download.oracle.com/otn_software/linux/instantclient/2112000/instantclient-basiclite-linux.x64-21.12.0.0.0dbru.zip
wget https://download.oracle.com/otn_software/linux/instantclient/2112000/instantclient-sdk-linux.x64-21.12.0.0.0dbru.zip
    
apt-get install unzip  
unzip instantclient-basiclite-linux.x64-21.12.0.0.0dbru.zip
unzip instantclient-sdk-linux.x64-21.12.0.0.0dbru.zip

sh -c "echo /opt/oracle/instantclient_21_12 > \
      /etc/ld.so.conf.d/oracle-instantclient.conf"

ldconfig

apt-get install php-pear
apt-get install -y php-dev
pecl install oci8-3.2.1
```

```
'instantclient,/opt/oracle/instantclient_21_12'
```

If you get error "Use PHP OCI8..." Check here:

[oci8 3.2.0](https://pecl.php.net/package/oci8/3.2.0)

```bash
cd /usr/lib/php/20210902/

echo "extension = oci8.so" >> /etc/php/8.1/cli/php.ini
echo "extension = oci8.so" >> /etc/php/8.1/apache2/php.ini
```

Now reboot your webserver instance:

```bash
reboot
```

Check info.php in your browser for OCI configuration details.

In the Oracle Cloud console, go to your Database details -->> Database connection and download your wallet.

Then on your Ubuntu client extract your wallet and change into the wallet directory.

Edit sqlnet.ora

The text in the file should look like this:

```
WALLET_LOCATION = (SOURCE = (METHOD = file) (METHOD_DATA = (DIRECTORY="/opt/wallet")))
SSL_SERVER_DN_MATCH=yes
```

```bash
# IMPORTANT! Make sure you are in your wallet directory on your client
sftp -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
mkdir wallet
cd wallet
put -r .
exit
```

```bash
ssh -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
sudo su
mv wallet /opt/
exit
```

Copy the PHP code below into a file oci8.php updating your credentials and database address:

```php
<?php

error_reporting(E_ALL);
ini_set('display_errors', 'On');

$conn = oci_connect('USERNAME', 'PASSWORD', 'tcps://adb.ap-melbourne-1.oraclecloud.com:1522/*******_harleys_high.adb.oraclecloud.com?wallet_location=/opt/wallet');

if (!$conn) {
    $e = oci_error();
    trigger_error(htmlentities($e['message'] ?? ''), E_USER_ERROR);
}
var_dump($conn);


// Parse the statement. Note there is no final semi-colon in the SQL statement
// https://www.php.net/manual/en/function.oci-parse.php
$stid = oci_parse($conn, 'SELECT * FROM ADMIN.ACCOUNTS');
if (!$stid) {
    $e = oci_error($conn);
    trigger_error(htmlentities($e['message'] ?? ''), E_USER_ERROR);
}
var_dump($stid);


// Define variables to store column values
oci_define_by_name($stid, 'USER_ID', $column1);
oci_define_by_name($stid, 'EMAIL', $column2);
oci_define_by_name($stid, 'PASSWORD', $column3);

oci_execute($stid);

// https://www.php.net/manual/en/function.oci-fetch-array.php
// Fetch and display results
echo "<table border='1'>\n";
while (oci_fetch($stid)) {
    echo "<tr>\n";
    echo "    <td>" . htmlentities($column1 ?? '') . "</td>\n";
    echo "    <td>" . htmlentities($column2 ?? '') . "</td>\n";
    echo "    <td>" . htmlentities($column3 ?? '') . "</td>\n";
    echo "</tr>\n";
}
echo "</table>\n";

oci_free_statement($stid);
oci_close($conn);

?>
```

```bash
sftp -i <ssh-private-key-path> ubuntu@<your-public-ip-address>
put oci8.php
exit
```

```bash
sudo su
chown root:root oci8.php
chmod 644 oci8.php
mv oci8.php /var/www/html/
exit
```

```
http://<your-public-ip-address>/oci8.php
```

# Destroy Resources

In [ ]:
!(cd tf-compute && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-database && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-vcn && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-compartment && tofu destroy -input=false -auto-approve)

In [ ]:
!(cd tf-provider && tofu destroy -input=false -auto-approve)